In [1]:
#
# Cell 1 (Modified)
#
%matplotlib inline 
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
# We need this to display the video in the notebook
from IPython.display import HTML

# Define arm parameters
L1 = 1.0  # Length of link 1
L2 = 1.0  # Length of link 2

In [2]:
#
# Cell 2: Forward Kinematics
#
def forward_kinematics(theta1, theta2):
    """
    Calculates the (x, y) position of the elbow and the hand.
    Angles are in radians.
    """
    x1 = L1 * np.cos(theta1)
    y1 = L1 * np.sin(theta1)
    
    x2 = x1 + L2 * np.cos(theta1 + theta2)
    y2 = y1 + L2 * np.sin(theta1 + theta2)
    
    return x1, y1, x2, y2

In [3]:
#
# Cell 3: Inverse Kinematics
#
def inverse_kinematics(x, y):
    """
    Calculates the joint angles (theta1, theta2) for a target (x, y) position.
    Returns two solutions for 'elbow up' and 'elbow down' configurations.
    Returns (None, None) if the target is unreachable.
    """
    # Check if the target is reachable
    r_squared = x**2 + y**2
    if r_squared > (L1 + L2)**2 or r_squared < (L1 - L2)**2:
        # Returning the last known valid angles could be an option for robustness
        return None, None

    # Calculate theta2 using the Law of Cosines
    cos_theta2 = (r_squared - L1**2 - L2**2) / (2 * L1 * L2)
    
    # Ensure the value is within arccos's valid range [-1, 1] due to potential float errors
    cos_theta2 = np.clip(cos_theta2, -1.0, 1.0)
    
    # Solution 1: Elbow down
    theta2_sol1 = -np.arccos(cos_theta2)
    
    # Solution 2: Elbow up
    theta2_sol2 = np.arccos(cos_theta2)

    # Calculate theta1 for both solutions using atan2 for robustness
    k1_sol1 = L1 + L2 * np.cos(theta2_sol1)
    k2_sol1 = L2 * np.sin(theta2_sol1)
    theta1_sol1 = np.arctan2(y, x) - np.arctan2(k2_sol1, k1_sol1)

    k1_sol2 = L1 + L2 * np.cos(theta2_sol2)
    k2_sol2 = L2 * np.sin(theta2_sol2)
    theta1_sol2 = np.arctan2(y, x) - np.arctan2(k2_sol2, k1_sol2)
    
    # Return both solutions in radians
    solution1 = (theta1_sol1, theta2_sol1)
    solution2 = (theta1_sol2, theta2_sol2)
    
    return solution1, solution2

In [4]:
#
# Cell 4: Trajectory Planning
#
def plan_trajectory(start_pos, end_pos, num_points=100):
    """
    Generates a straight-line trajectory in Cartesian space and 
    calculates the corresponding joint angles.
    """
    x_path = np.linspace(start_pos[0], end_pos[0], num_points)
    y_path = np.linspace(start_pos[1], end_pos[1], num_points)

    joint_angles_path = []
    for x, y in zip(x_path, y_path):
        # Consistently use the 'elbow up' solution (solution 2) for a smooth path
        _ , elbow_up_solution = inverse_kinematics(x, y)
        if elbow_up_solution is not None:
            joint_angles_path.append(elbow_up_solution)
        else:
            print(f"Warning: Point ({x:.2f}, {y:.2f}) in trajectory is unreachable.")
            if joint_angles_path: # If a point is unreachable, hold the last valid position
                joint_angles_path.append(joint_angles_path[-1])
            else: # If the very first point is unreachable, this will fail. Add a fallback.
                return None 

    return np.array(joint_angles_path)

# --- Define start and end points for the trajectory ---
start_pos = (1.5, 0.5)
end_pos = (-0.5, 1.2)

# Plan the trajectory
trajectory = plan_trajectory(start_pos, end_pos)

# Check if trajectory was created successfully
if trajectory is not None:
    print(f"Successfully planned trajectory with {len(trajectory)} points.")

Successfully planned trajectory with 100 points.


In [5]:
# Cell 5 (Final Version: Saves GIF and Displays Video)

if trajectory is not None:
    # Setup the plot (This part is unchanged)
    fig, ax = plt.subplots()
    ax.set_xlim(-2.5, 2.5)
    ax.set_ylim(-2.5, 2.5)
    ax.set_aspect('equal')
    ax.grid()

    line, = ax.plot([], [], 'o-', lw=3, markersize=8, color='#0066CC')
    
    def init():
        line.set_data([], [])
        return line,

    def animate(i):
        theta1, theta2 = trajectory[i]
        x1, y1, x2, y2 = forward_kinematics(theta1, theta2)
        line.set_data([0, x1, x2], [0, y1, y2])
        return line,

    # Create the animation object (This part is unchanged)
    ani = FuncAnimation(fig, animate, frames=len(trajectory),
                        init_func=init, blit=True, interval=20)

    # --- ACTION REQUIRED HERE ---

    # 1. SAVE the animation as a GIF file
    print("Saving animation as a GIF... This may take a moment.")
    ani.save('robotic_arm_trajectory.gif', writer='imagemagick', fps=30)
    print("GIF saved successfully!")

    # 2. DISPLAY the animation as an HTML5 video in the notebook
    display(HTML(ani.to_html5_video()))
    
    # 3. CLOSE the plot to prevent an empty extra plot from showing up
    plt.close()
    
    # --------------------------
    
else:
    print("Cannot create animation because trajectory planning failed.")

MovieWriter imagemagick unavailable; using Pillow instead.


Saving animation as a GIF... This may take a moment.
GIF saved successfully!
